# 40 - Extraction & Triage: produce structured report and assign specialist


In [4]:
# Assume chat_history variable exists or paste your transcript (string)
# Example:
chat_history = "Patient: I have chest pain since morning. Bot: ... Patient: The pain is sharp, dizzy, vomit sometimes."
print(chat_history)


Patient: I have chest pain since morning. Bot: ... Patient: The pain is sharp, dizzy, vomit sometimes.


In [2]:
import requests, json, os

OLLAMA_URL = os.getenv("OLLAMA_URL","http://127.0.0.1:11434")
MODEL_NAME = os.getenv("MODEL_NAME","llama3")

def call_ollama(prompt, max_tokens=300, temperature=0.0, timeout=60):
    """
    Handles Ollama NDJSON streaming output.
    Always returns a single concatenated string containing the model response.
    """
    url = f"{OLLAMA_URL.rstrip('/')}/api/generate"
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stream": False    # enforce non-stream mode
    }

    # NEW: Disable streaming so we get a single JSON object
    r = requests.post(url, json=payload, timeout=timeout)

    # If server STILL streams NDJSON, parse line-by-line manually:
    text = r.text.strip()

    # Try simple JSON parse first (if server honors stream=False)
    try:
        data = r.json()
        if "response" in data:
            return data["response"]
        if "output" in data:
            return data["output"]
    except:
        pass  # fallback to NDJSON parsing below

    # NDJSON fallback
    lines = text.split("\n")
    output = ""
    for line in lines:
        try:
            obj = json.loads(line)
            if "response" in obj and obj["response"]:
                output += obj["response"]
        except:
            continue

    return output.strip()


In [5]:
# Ask LLM to emit strict JSON per schema
schema = """
Output ONLY a single valid JSON object with keys:
- symptoms: list of strings
- duration: string
- severity: string (mild/moderate/severe)
- medications: list
- allergies: list
- urgency: string (low/medium/high)
Do not output any extra text.
"""
prompt = f"""{schema}

Conversation:
{chat_history}

Now output ONLY the JSON.
"""
raw = call_ollama(prompt, max_tokens=500)
print("Raw output:\n", raw[:1000])


Raw output:
 {
"symptoms": ["chest pain", "sharp pain", "dizziness", "vomiting"],
"duration": "morning",
"severity": "severe",
"medications": [],
"allergies": [],
"urgency": "high"
}


In [6]:
# Extract JSON robustly
import re, json
m = re.search(r'(\{.*\})', raw, re.S)
if not m:
    print("No JSON found. Raw starts:", raw[:400])
    structured = {}
else:
    structured = json.loads(m.group(1))
    print("Structured:", structured)


Structured: {'symptoms': ['chest pain', 'sharp pain', 'dizziness', 'vomiting'], 'duration': 'morning', 'severity': 'severe', 'medications': [], 'allergies': [], 'urgency': 'high'}


In [7]:
# Simple rule-based triage
def triage_report(s):
    symptoms = " ".join(s.get("symptoms", [])).lower()
    if any(w in symptoms for w in ["chest","breath","shortness","palpit"]):
        return {"specialist":"Cardiology/Emergency","urgency":"high"}
    if any(w in symptoms for w in ["rash","itch","lesion"]):
        return {"specialist":"Dermatology","urgency":"medium"}
    if any(w in symptoms for w in ["headache","dizzy","seizure"]):
        return {"specialist":"Neurology","urgency":"medium"}
    return {"specialist":"General Physician","urgency":"low"}

if structured:
    tri = triage_report(structured)
    print("Triage:", tri)
else:
    print("No structured report to triage.")



Triage: {'specialist': 'Cardiology/Emergency', 'urgency': 'high'}


In [8]:
# Save report locally (or send to Mongo later)
out = {"chat_history": chat_history, "structured": structured, "triage": tri}
import json, os
os.makedirs("outputs", exist_ok=True)
with open("outputs/report.json","w",encoding="utf-8") as f:
    json.dump(out, f, indent=2)
print("Saved: outputs/report.json")


Saved: outputs/report.json
